# Cohort # 1 MLOPs-Zoomcap 2023

Load of libraries and modules needed

In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

Download 2022 january and february .PARQUET files

In [ ]:
!wget -N -v --no-check-certificate https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!wget -N -v --no-check-certificate https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

### Q1
**Read the data for January. How many columns are there?**

Loading january data from .PARQUET file

In [ ]:
df_train = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df_train.head()

In [ ]:
df_train.shape

There are 19 columns in the dataframe

Computing duration in minutes

In [ ]:
# df_train['duration'] equals diference between pickup and dropoff in seconds
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
# Convert df_train['duration'] to minutes
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds()/60)

### Q2
**What's the standard deviation of the trips duration in January?**

In [ ]:
df_train['duration'].describe()

The standar deviation is 46.445 minutes

Dropping outliers

In [ ]:
# filter only duration over 1 minutes and less than 1 hour
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

### Q3
**What fraction of the records left after you dropped the outliers?**

In [ ]:
df_train.shape

In [ ]:
round(2421440/2463931*100,2)

One hot encoding

In [ ]:
# select the features to train the model
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
# convert the datatype of categorical features from integer to string
df_train[categorical] = df_train[categorical].astype(str)

In [ ]:
# convert the dataframe into a dictionary
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [ ]:
# create a DictVectorizer and transform the lists of feature-value mappings to vectors
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

### Q4
**What's the dimensionality of this matrix (number of columns)?**

In [ ]:
X_train.shape

Training a model

In [ ]:
# select the target feature for train the model
target = 'duration'
y_train = df_train[target].values

In [ ]:
# define the model as LinearRegression class
lr = LinearRegression()
# train the model
lr.fit(X_train, y_train)

In [ ]:
# test prediction with X_train data
y_pred = lr.predict(X_train)

### Q5
**What's the RMSE on train?**

In [ ]:
# mean squared value between predicted values and y_pred values
round(mean_squared_error(y_train, y_pred, squared=False),2)

Evaluating the model

In [ ]:
df_val = pd.read_parquet('yellow_tripdata_2022-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds()/60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = df_val[target].values

In [ ]:
y_pred = lr.predict(X_val)

### Q6
**What's the RMSE on validation?**

In [ ]:
round(mean_squared_error(y_val, lr.predict(X_val), squared=False),3)